In [1]:
import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import math

In [2]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：1



,No,Url
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://www.dormanproducts.com/gsearch.aspx?ty...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param.pop('start', None)
dict_param.pop('num', None)

url_request = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())
            if count > 0:
                with open('./page.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

count

1
2
3
4
5


165

In [5]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Url': url_request,
                          'SKU Count': count,
                          'Page': math.ceil(count / 100)})

list_dict_correct

[{'No': 1,
  'Url': 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=engine+timing+cover',
  'SKU Count': 165,
  'Page': 2}]

In [6]:
crawler_status = True

crawler_status

True

In [7]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.dormanproducts.com/gsearch.aspx?ty...
 Request_Url    https://www.dormanproducts.com/gsearch.aspx?ty...
 Name: 0, dtype: object]

In [8]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_page.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Url,SKU Count,Page
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,165,2


In [9]:
print('Output Finished !')
print()

Output Finished !



In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_page_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,https://www.dormanproducts.com/gsearch.aspx?ty...


In [11]:
print('Done ~')

Done ~
